# **Regrid Satellite Data**
Code written by: Billy Atkinson (watkinson@umces.edu) \
Date of last revision: 08/03/2022 \
This code takes in Satellite Data from MODIS Aqua and regrids it to match the grid used for the HYCOM Absolute Dynamic Topography (ADT) data from the Gulf of Mexico Reanalysis Experiment 50.1  \
The HYCOM ADT data is useful for detecting mesoscale eddies in the Gulf of Mexico \
Documentation: [XESMF Documentation](https://xesmf.readthedocs.io/en/latest/notebooks/Dataset.html)

In [1]:
import xarray as xr 
import numpy as np
import pandas as pd
import xesmf as xe
import os
import glob

The while statement below is used to adjust the longitude for each satellite data file and regird each file to match the ADT grid. \
Each resulting file is saved to a new netcdf data format file.

In [2]:
dir = '/data/watkinson/Fall2022/Sat_data/1day_Original/'
ds = xr.open_dataset(dir + 'A2002198.L4_gom_day_CAFE.nc')
print(ds.lon)

<xarray.DataArray 'lon' (lon: 2112)>
array([-97.99479 , -97.984375, -97.97395 , ..., -76.02604 , -76.01562 ,
       -76.0052  ], dtype=float32)
Coordinates:
    time     datetime64[ns] ...
  * lon      (lon) float32 -97.99 -97.98 -97.97 -97.96 ... -76.03 -76.02 -76.01
Attributes:
    long_name:      Longitude
    units:          degrees_east
    standard_name:  longitude
    valid_min:      -180.0
    valid_max:      180.0


In [ ]:
sat_file_objects = sorted(glob.glob('/data/watkinson/Fall2022/Sat_data/1day_regrid/A*.nc'))

In [30]:
sat_file_objects = sorted(glob.glob('/data/watkinson/Fall2022/Sat_data/1day_Original/A*.nc'))
sat_file_objects = sat_file_objects[490:1630]
#print(len(sat_file_objects))
#print(np.shape(sat_file_objects))
#print(sat_file_objects.index('/data/watkinson/Fall2022/Sat_data/1day_Original/A2006365.L4_gom_day_CAFE.nc'))
#print(sat_file_objects)
#ssh_file_objects = sorted(glob.glob("/data/watkinson/Summer2022/data/hycom_ssh/Daily_SSH_files/hycom_gomu_501_2006*.nc"))
#print(np.shape(ssh_file_objects))
hycom_in = sorted(glob.glob('/data/watkinson/Fall2022/SSH_data/Cropped_SSH_Data/hycom_gomu_501_*.nc'))
#print(hycom_in.index('/data/watkinson/Fall2022/SSH_data/Cropped_SSH_Data/hycom_gomu_501_20061231_cropped.nc'))
#print(len(hycom_in))
hycom_in = hycom_in[499:1639]
#print(len(hycom_in))
#date = os.path.basename(hycom_file)[15:23]
#dir = "/data/watkinson/Summer2022/data/cafe/1day/"
#sat_file_names = sorted(os.listdir(dir))
#sat_file_names=sat_file_names[537:903]

n=0
while n < len(sat_file_objects):   
    date = os.path.basename(hycom_in[n])[15:23]
    ds = xr.open_dataset(sat_file_objects[n])
    #ds['lon']=ds.lon + 360
    ds_out = xr.open_dataset(hycom_in[n])
    regridder = xe.Regridder(ds,ds_out,"bilinear")
    ds_regrid = regridder(ds)
    dir = '/data/watkinson/Fall2022/Sat_data/1day_regrid/'
    ds_regrid.to_netcdf(dir + 'CAFE_GOM_day_'+ date,mode='w')
    n=n+1


In [10]:
#dl = xr.open_dataset('/data/watkinson/Fall2022/Sat_data/1day_regrid/CAFE_GOM_day_20030101')
#dl
dx = xr.open_dataset('/data/watkinson/Fall2022/SSH_data/Cropped_SSH_Data/hycom_gomu_501_20030101_cropped.nc')
dx

<xarray.Dataset>
Dimensions:  (time: 1, lon: 541, lat: 346)
Coordinates:
  * time     (time) datetime64[ns] 2003-01-01
  * lon      (lon) float64 -98.0 -97.96 -97.92 -97.88 ... -76.48 -76.44 -76.4
  * lat      (lat) float64 18.12 18.16 18.2 18.24 ... 31.8 31.84 31.88 31.92
Data variables:
    adt      (lat, lon, time) float32 ...
    ugos     (lat, lon, time) float32 ...
    vgos     (lat, lon, time) float32 ...